In [1]:
import requests
from urllib.parse import quote
import pandas as pd
from bs4 import BeautifulSoup

In [30]:
url = 'https://www.genie.co.kr/chart/top200'
result = requests.get(url)
result.text

'<br>\r\n<br>\r\n<center>\r\n<img src="http://www.geniemusic.co.kr/images/common/logo_r1.png"><br>\r\n<h2> <meta http-equiv="Content-Type" content="text/html;charset=UTF-8"> ì\xa0\x91ì\x86\x8dì\x9a\x94ì²\xadì\x9d´ ë³´ì\x95\x88ì\xa0\x95ì±\x85ì\x97\x90 ì\x9d\x98í\x95´ ì°¨ë\x8b¨ë\x90\x98ì\x97\x88ì\x8aµë\x8b\x88ë\x8b¤. ë\x8b¹ì\x82¬ ê³\xa0ê°\x9dì\x84¼í\x84°ë¡\x9c ë¬¸ì\x9d\x98í\x95´ì£¼ì\x8b\xadì\x8b\x9cì\x98¤.<br><br>\r\nThe security policy of the connection request is blocked. Contact your customer service representative.<br><br>\r\nì§\x80ë\x8b\x88ë®¤ì§\x81 ê³\xa0ê°\x9dì\x84¼í\x84° 1577-5337<br><br>\r\n</h2>\r\n</center>\r\n<br>'

In [ ]:
header = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome'}
result = requests.get(url, header=header)
soup = BeautifulSoup(result.text, 'html.parser')

#### 2. 원하는 데이터 찾기

In [14]:
trs = soup.select('tr.list')
len(trs)


0

In [15]:
# 개발자 도구에서 찾아주는 Selector
# #body-content > div.newest-list > div > table > tbody > tr
trs = soup.select('#body-content > div.newest-list > div > table > tbody > tr')
len(trs)

0

##### 3. 하나를 선택해서 원하는 정보 추출

In [ ]:
tr = trs[0]

In [ ]:
# rank
tr.select_one('.number').get_text()

In [ ]:
rank = tr.select_one('.number').get_text().split('\n')[0].strip()

In [ ]:
# img url
img = 'https:' + tr.select_one('.cover > img')['src']
img

In [ ]:
# title
tr.select_one('.title.ellipsis').get_text().strip()

In [ ]:
# 19금 노래 제목
tr.select_one('.title.ellipsis').get_text().split('\n')[-1].strip()

In [ ]:
#위의 두 코드를 합치면
# 방법 1)
title = tr.select_one('.title.ellipsis').get_text().split('\n')[-1].strip()
# 방법 2)
try:
    title = tr.select_one('.title.ellipsis').get_text().split()
except:
    title = tr.select_one('.title.ellipsis').get_text().split('\n')[-1].strip()

In [ ]:
# artist
artist = tr.select_one('.artitst.ellipsis').string.strip()
artist

In [ ]:
# album
album = tr.select_one('.albumtitle.ellipsis').string.strip()
album

##### 4. 한 페이지에 있는 정보

In [ ]:
#방법 1. 딕셔너리의 리스트로 가져오기
line = []
for tr in trs:
    rank = tr.select_one('number').string.strip()
    img = 'https:' + tr.select_one('.cover > img')['src']
    title = tr.select_one('.artitst.ellipsis').string.strip()
    artist = tr.select_one('.artitst.ellipsis').string.strip()
    album = tr.select_one('.albumtitle.ellipsis').string.strip()
    line.append({'rank':rank, 'img':img, 'title':title, 'artist':artist, 'album':album})


In [ ]:
# 방법 2. 딕셔너리의 리스트로 가져오기
rank_list, img_list, title_list, artist_list, album_list = [], [], [], [], []
for tr in trs:
    rank_list.append(tr.select_one('number').string.strip())
    img_list.append('https:' + tr.select_one('.cover > img')['src'])
    title_list.append(tr.select_one('.artitst.ellipsis').string.strip())
    artist_list.append(tr.select_one('.artitst.ellipsis').string.strip())
    album_list.append(tr.select_one('.albumtitle.ellipsis').string.strip())
    line.append({'rank':rank, 'img':img, 'title':title, 'artist':artist, 'album':album})


##### 5. 에러처리

##### 6. 모든 페이지에 대해서 처리하기

In [21]:
# https://www.genie.co.kr/chart/top200?ditc=D&ymd=20230424&hh=15&rtm=Y&pg=1

from datetime import datetime
now = datetime.now()
now

datetime.datetime(2023, 4, 24, 15, 4, 28, 875003)

In [24]:
ymd = now.strftime('%Y%m%d')         # ymd = f'{now.get.year}
hh = now.strftime('%H')
ymd, hh

('20230424', '15')

In [ ]:
from tqdm import tqdm
lines = []
base_url : 'https://www.genie.co.kr/chart/top200?ditc=D&rtm=Y'
for page in tqdm(range(1, 5)):
    url = f'{base_url}&ymd={ymd}&hh={hh}&pg={page}'
    result = requests.get(url, header=header)
    soup = BeautifulSoup(result.text, 'html.parser')
    trs = soup.select('tr.list')
    for tr in trs:
        rank = tr.select_one('number').string.strip()
        img = 'https:' + tr.select_one('.cover > img')['src']
        title = tr.select_one('.artitst.ellipsis').string.strip()
        artist = tr.select_one('.artitst.ellipsis').string.strip()
        album = tr.select_one('.albumtitle.ellipsis').string.strip()
        line.append({'rank':rank, 'img':img, 'title':title, 'artist':artist, 'album':album})

##### 7. 데이터프레임 만들기

In [31]:
df = pd.DataFrame(lines)
df.head()

""


In [39]:
df.to_csv(f'data/{ymd}{hh}_지니차트.csv', index=False)